# Building Machine Learning Classifiers: Building a basic Random Forest model

### Read in & clean text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Explore RandomForestClassifier Attributes & Hyperparameters

In [2]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_feature_names', '_check_n_features', '_compute_oob_predictions', '_estimator_type', '_get_oob_predictions', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_set_oob_score_and_attributes', '_validate_X_predict', '_validate_data', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'n_features_', 'predict', 'predict_log_proba', 'predict_proba', 'score',

### Explore RandomForestClassifier through Cross-Validation

In [4]:
from sklearn.model_selection import KFold, cross_val_score

In [5]:
rf = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf, X_features, data['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

array([0.97845601, 0.98025135, 0.97484277, 0.96585804, 0.97574124])

### Explore RandomForestClassifier through Holdout Set

In [16]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [18]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)

c:\users\raulr\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [19]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[0:10]

[(0.05065431957797225, 'body_len'),
 (0.042965095443710355, 1803),
 (0.02839478534155264, 7350),
 (0.025304390513870866, 3134),
 (0.02464571094462807, 4796),
 (0.022467993102521323, 7027),
 (0.017751039606520694, 2031),
 (0.01721183138807117, 7461),
 (0.016153822797425537, 5724),
 (0.014759748179728869, 397)]

In [20]:
y_pred = rf_model.predict(X_test)

precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

c:\users\raulr\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [21]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision,3),
                                                        round(recall,3),
                                                        round((y_pred==y_test).sum() / len(y_pred),3)))

Precision: 1.0 / Recall: 0.59 / Accuracy: 0.949


### Build our own Grid-search

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [23]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} ------ Precision: {} / Recall: {} / Accuracy: {}'.format(n_est, depth,
                                                                                        round(precision,3),
                                                                                        round(recall,3),
                                                                                        round((y_pred==y_test).sum() / len(y_pred),3)))

In [26]:
import warnings
warnings.filterwarnings("ignore")
for n_est in [10,50,100]:
    for depth in [10,20,30,None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 ------ Precision: 1.0 / Recall: 0.424 / Accuracy: 0.922
Est: 10 / Depth: 20 ------ Precision: 1.0 / Recall: 0.55 / Accuracy: 0.939
Est: 10 / Depth: 30 ------ Precision: 0.99 / Recall: 0.689 / Accuracy: 0.957
Est: 10 / Depth: None ------ Precision: 1.0 / Recall: 0.762 / Accuracy: 0.968
Est: 50 / Depth: 10 ------ Precision: 1.0 / Recall: 0.212 / Accuracy: 0.893
Est: 50 / Depth: 20 ------ Precision: 1.0 / Recall: 0.616 / Accuracy: 0.948
Est: 50 / Depth: 30 ------ Precision: 1.0 / Recall: 0.709 / Accuracy: 0.961
Est: 50 / Depth: None ------ Precision: 1.0 / Recall: 0.821 / Accuracy: 0.976
Est: 100 / Depth: 10 ------ Precision: 1.0 / Recall: 0.225 / Accuracy: 0.895
Est: 100 / Depth: 20 ------ Precision: 1.0 / Recall: 0.589 / Accuracy: 0.944
Est: 100 / Depth: 30 ------ Precision: 1.0 / Recall: 0.715 / Accuracy: 0.961
Est: 100 / Depth: None ------ Precision: 1.0 / Recall: 0.828 / Accuracy: 0.977


### Exploring parameter settings using GridSearchCV

In [27]:
from sklearn.model_selection import GridSearchCV

In [30]:
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
rf = RandomForestClassifier()
param = {'n_estimators': [10,150,300],
         'max_depth':[30,60,90,None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,4.986674,3.066279,0.339476,0.099169,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.977558,0.982944,0.974843,0.964960,0.973046,0.974670,0.005895,1
7,23.114834,0.300837,0.342678,0.076729,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.975763,0.978456,0.975741,0.968553,0.973046,0.974312,0.003349,2
8,42.345184,0.534825,0.339478,0.026453,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.976661,0.975741,0.968553,0.970350,0.973773,0.003619,3
10,22.586512,0.438293,0.210848,0.026827,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977558,0.976661,0.973944,0.967655,0.969452,0.973054,0.003904,4
11,33.427486,0.683834,0.211653,0.015413,None,300,"{'max_depth': None, 'n_estimators': 300}",0.978456,0.974865,0.974843,0.967655,0.968553,0.972875,0.004121,5


#### Count Vectorizer

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])

X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

In [32]:
rf = RandomForestClassifier()
param = {'n_estimators': [10,150,300],
         'max_depth':[30,60,90,None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,39.207756,0.896548,0.294867,0.045264,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976661,0.975763,0.973944,0.968553,0.970350,0.973054,0.003121,1
10,27.475938,0.762783,0.286265,0.040085,None,150,"{'max_depth': None, 'n_estimators': 150}",0.978456,0.973070,0.973046,0.966757,0.972147,0.972695,0.003715,2
8,50.043125,0.810444,0.426096,0.028946,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.973070,0.974843,0.967655,0.968553,0.972336,0.003750,3
7,25.794314,1.663103,0.382089,0.074267,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.979354,0.973968,0.973944,0.964960,0.967655,0.971976,0.005102,4
6,3.445980,0.293791,0.179641,0.041764,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.977558,0.974865,0.973046,0.964061,0.969452,0.971797,0.004680,5
